In [1]:
import os
import sys
# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)

In [2]:
from functions import write_dataframe_to_excel
from functions import read_and_check_input, Separate_data, nettoyer_dataframe
from functions import construire_tableau, Transpose_dataframe
from functions import format_constraints_elements, format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import construct_result_dataframe,Save_errors

In [3]:
def create_optimal_recipe(chemin_fichier):
    # Lire les fichiers Excel
    df_table, df_MP, df_contraints, erreurs = read_and_check_input(chemin_fichier)

    if len(erreurs) != 0 :
        return 0
    df_contraints_element, df_contraints_qualite, df_MP_dispo, df_MP_indispo = Separate_data(df_table, df_MP, df_contraints)

    # Suppression des matières premières indisponibles
    df_table = nettoyer_dataframe(df_table, df_MP_indispo)

    # Construction de la matrice A et du vecteur C
    A, C = construire_tableau(df_table, df_MP_dispo)

    # Initialisation des listes pour les contraintes
    A_ub, b_ub, A_eq, b_eq = [], [], [], []

    df_contraints_element = Transpose_dataframe(df_contraints_element)
    # Mettre  les contraintes concernant les éléments
    A_ub, b_ub, A_eq, b_eq = format_constraints_elements(A_ub, b_ub, A_eq, b_eq, df_contraints_element, A)

    # Mettre  les contraintes concernant la qualité
    A_ub, b_ub, A_eq, b_eq = format_constraints_qualite(A_ub, b_ub, A_eq, b_eq, df_contraints_qualite, A)

    # Mettre les contraintes concernant les matières premières disponibles
    A_eq, b_eq, bounds = format_constraints_MP(A_eq, b_eq, df_MP_dispo)

    # Résoudre le problème d'optimisation linéaire
    res = solve_linear_program(C, A_ub, b_ub, A_eq, b_eq, bounds)

    # On recupere le chemin du dossier data
    dossier_data = os.path.dirname(chemin_fichier)

    # Gestion du résultats
    if res.success:
        # Construire le DataFrame résultats
        df_res = construct_result_dataframe(df_MP_dispo, df_table, res)
        # Écrire le DataFrame résultats dans le fichier Excel
        write_dataframe_to_excel(df_res, dossier_data, new_sheet_name='Recette')
        print("Le problème admet une solution optimale.")

    else :
        # Renvoie le message d'erreur
        message = "Erreur : Veillez revoir les contraintes de la feuille 'Contraintes Qualités et Elément' et/ou 'Contraintes matières premières'"
        Save_errors(message, dossier_data)
        print("Les erreurs ont été enregistrées dans le fichier erreurs.txt")
    return res


In [4]:
if __name__ == "__main__":
    # Chemin du fichier
    chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
    create_optimal_recipe(chemin_fichier)

Les erreurs ont été enregistrées dans le fichier erreurs.txt


#### Essais


In [4]:
import os
import sys
# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)

from functions import write_dataframe_to_excel
from functions import read_and_check_input, Separate_data, nettoyer_dataframe
from functions import construire_tableau, Transpose_dataframe
from functions import format_constraints_elements, format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import construct_result_dataframe,Save_errors
import os

In [5]:
chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')


# Lire les fichiers Excel
df_table, df_MP, df_contraints, erreurs = read_and_check_input(chemin_fichier)

df_contraints_element, df_contraints_qualite, df_MP_dispo, df_MP_indispo = Separate_data(df_table, df_MP, df_contraints)

# Suppression des matières premières indisponibles
df_table = nettoyer_dataframe(df_table, df_MP_indispo)

# Construction de la matrice A et du vecteur C
A, C = construire_tableau(df_table, df_MP_dispo)

# Initialisation des listes pour les contraintes
A_ub, b_ub, A_eq, b_eq = [], [], [], []

df_contraints_element = Transpose_dataframe(df_contraints_element)
# Mettre  les contraintes concernant les éléments
A_ub, b_ub, A_eq, b_eq = format_constraints_elements(A_ub, b_ub, A_eq, b_eq, df_contraints_element, A)

# Mettre  les contraintes concernant la qualité
A_ub, b_ub, A_eq, b_eq = format_constraints_qualite(A_ub, b_ub, A_eq, b_eq, df_contraints_qualite, A)

# Mettre les contraintes concernant les matières premières disponibles
A_eq, b_eq, bounds = format_constraints_MP(A_eq, b_eq, df_MP_dispo)


from scipy.optimize import linprog


In [6]:
def check_contraints_conforme(residuals, tol=1e-5):
    result = []
    for val in residuals:
        if val > 0:
            result.append(1)
        elif abs(val) <= tol: 
            result.append(1)
        else:
            result.append(0)
    conforme = all(val == 1 for val in result)
    return conforme, result

In [7]:
def solve():
    LINPROG_METHODS = ['revised simplex', 'interior-point']
    for method in LINPROG_METHODS :
        # On resoudre d'abord avec cette methode le problème
        res  = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,bounds=bounds,  method ='highs')
        if res.success :
            print(res.status,res.nit,res.x)
            return res
        
        if res.status != 0 :
            # Essaie d'autres methodes de la bibliothèque 
            res  = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,bounds=bounds,method = method, options={"presolve": False}) #False
            ce_residuals, ci_residuals = res.con, res.slack
            conforme_ce, result_ce = check_contraints_conforme(ce_residuals, tol=1e-5)
            conforme_ci, result_ci = check_contraints_conforme(ci_residuals, tol=1e-5)
            if conforme_ce and  conforme_ci :
                return  res 


In [8]:

LINPROG_METHODS = ['simplex', 'revised simplex', 'interior-point', 'highs', 'highs-ds', 'highs-ipm']
LINPROG_METHODS = ['simplex', 'revised simplex', 'interior-point']
LINPROG_METHODS = ['revised simplex', 'interior-point']
for method in LINPROG_METHODS :
    # Résoudre le problème d'optimisation linéaire
    res  = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,bounds=bounds,  method =method)
    if res.success :
        print(res.status,res.nit,res.x)
        break;

    if res.status != 0 :
        # Essaie d'autres methodes de la bibliothèque 
        res  = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,bounds=bounds,method =method, options={"presolve": False}) #False
        ce_residuals, ci_residuals = res.con, res.slack
        conforme_ce, result_ce = check_contraints_conforme(ce_residuals, tol=1e-5)
        conforme_ci, result_ci = check_contraints_conforme(ci_residuals, tol=1e-5)
        if conforme_ce and  conforme_ci :
            # return  res 
            print(method, '\n')
            print(res.status,res.nit,res.x,res.fun)
            print(ci_residuals,ce_residuals)
        # break;



/tmp/ipykernel_28270/3438882317.py:6: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res  = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,bounds=bounds,  method =method)
/tmp/ipykernel_28270/3438882317.py:13: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res  = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,bounds=bounds,method =method, options={"presolve": False}) #False
/tmp/ipykernel_28270/3438882317.py:6: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res  = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,bounds=bounds,  method =method)


revised simplex 

2 22 [0.00000000e+00 0.00000000e+00 1.33468898e-01 3.08265551e-01
 1.14092106e-01 1.38881888e-02 1.20986689e-01 0.00000000e+00
 3.72007103e-02 1.44710467e-03 0.00000000e+00 1.56320034e-02
 0.00000000e+00 2.00209115e-04 1.32619033e-02] 408.5713752370013
[-4.44089210e-16  1.00000000e-01 -4.44089210e-16  3.00000000e-01
  5.55111512e-17  1.00000000e-01 -1.38777878e-17 -1.38777878e-17
 -1.73472348e-17  1.03963971e-02  2.28998741e-03  4.66869622e-03
  0.00000000e+00  1.11294019e-02 -3.46944695e-18  4.29288636e-02
  2.81917344e-02  8.96210907e-03  3.91029785e-03  2.16840434e-19
  1.30355139e-03  0.00000000e+00  1.20000000e-01  1.19378120e-02
  4.30621880e-02] [0.24155664 0.23590789]


/tmp/ipykernel_28270/3438882317.py:13: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res  = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,bounds=bounds,method =method, options={"presolve": False}) #False


In [9]:

import scipy

print(scipy.__version__)


1.11.4


In [10]:
def linprog_terse_callback(res):
    """
    A sample callback function demonstrating the linprog callback interface.
    This callback produces brief output to sys.stdout before each iteration
    and after the final iteration of the simplex algorithm.

    Parameters
    ----------
    res : A `scipy.optimize.OptimizeResult` consisting of the following fields:

        x : 1-D array
            The independent variable vector which optimizes the linear
            programming problem.
        fun : float
            Value of the objective function.
        success : bool
            True if the algorithm succeeded in finding an optimal solution.
        slack : 1-D array
            The values of the slack variables. Each slack variable corresponds
            to an inequality constraint. If the slack is zero, then the
            corresponding constraint is active.
        con : 1-D array
            The (nominally zero) residuals of the equality constraints, that is,
            ``b - A_eq @ x``.
        phase : int
            The phase of the optimization being executed. In phase 1 a basic
            feasible solution is sought and the T has an additional row
            representing an alternate objective function.
        status : int
            An integer representing the exit status of the optimization::

                 0 : Optimization terminated successfully
                 1 : Iteration limit reached
                 2 : Problem appears to be infeasible
                 3 : Problem appears to be unbounded
                 4 : Serious numerical difficulties encountered

        nit : int
            The number of iterations performed.
        message : str
            A string descriptor of the exit status of the optimization.
    """
    nit = res['nit']
    x = res['x']

    if nit == 0:
        print("Iter:   X:")
    print(f"{nit: <5d}   ", end="")
    print(x)